<a href="https://colab.research.google.com/github/the-jola-amodu/Llamaindex-RAG-using-Python/blob/main/Outamation_Section_5_Open_Source_Model_and_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries with CUDA support
!pip install -q torch
# Check CUDA version first
!nvcc --version

# Install llama-cpp-python with CUDA 12.x support
!pip install --no-cache-dir llama-cpp-python==0.2.90 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu123
!pip install llama-index

!pip install llama-index-llms-llama-cpp llama-index llama-index-embeddings-huggingface
!pip install -q llama-index-vector-stores-chroma
!pip install -q sentence-transformers
!pip install -q chromadb
!pip install nest_asyncio pymupdf
!pip install llama-index-retrievers-bm25

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu123
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.5/444.5 MB 215.4 MB/s eta 0:00:00
  Attempting uninstall: llama-cpp-python
    Found existing installation: llama_cpp_python 0.3.8
    Uninstalling llama_cpp_python-0.3.8:
      Successfully uninstalled llama_cpp_python-0.3.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-llms-llama-cpp 0.4.0 requires llama-cpp-python<0.4.0,>=0.3.0, but you have llama-cpp-python 0.2.90 which is incompatible.
  Using cached llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl
  Attempting uninstall: llama-cpp-python
 

In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA available: False


In [ ]:
import os

# Download Mistral model if not already present
model_path = "/content/mistral.gguf"
if not os.path.exists(model_path):
    # !wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf -O {model_path} # Mistral
    !wget https://huggingface.co/TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF/resolve/main/tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf -O {model_path} # TinyLlama

    print(f"Model downloaded to {model_path}")

# Verify file exists and check size
if os.path.exists(model_path):
    print(f"Model file exists. Size: {os.path.getsize(model_path) / (1024 * 1024):.2f} MB")
else:
    print("Model file not found!")


Model file exists. Size: 637.81 MB


In [ ]:
import os
import fitz
import nest_asyncio
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import QueryFusionRetriever
from llama_index.core import VectorStoreIndex
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core import Settings
from llama_index.core import Document
from llama_index.core.node_parser import SemanticSplitterNodeParser, SentenceSplitter
from typing import List
from google.colab import files
from llama_index.core import Document
from typing import List
import os

# Helper Functions to upload PDF, extract text, create Document objects and create vector indices
def upload_pdf():
    """Upload a PDF file and return its path."""
    print("Please select a PDF file to upload:")
    uploaded = files.upload()
    document_paths = []
    for filename in uploaded.keys():
        if filename.endswith('.pdf'):
            # Save to the sample_docs directory
            pdf_path = os.path.join("sample_docs", filename)

            # Create directory if it doesn't exist
            os.makedirs("sample_docs", exist_ok=True)

            # Save the file
            with open(pdf_path, 'wb') as f:
                f.write(uploaded[filename])

            print(f"PDF saved to {pdf_path}")
            document_paths.append(pdf_path)
        else:
            print(f"File {filename} is not a PDF. Please upload a PDF file.")

    return document_paths

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using PyMuPDF."""
    doc_texts = {}
    for i, path in enumerate(pdf_path):
        doc = fitz.open(path)

        # Extract text from all pages
        text = "\n".join([page.get_text() for page in doc])
        doc_texts[f"Doc-{i+1}"] = text

        # Print some stats
        print(f"PDF: {path}")
        print(f"Number of pages: {len(doc)}")
        print(f"Extracted {len(text.split())} words from the PDF.")

        # Close the document
        doc.close()

    return doc_texts


In [ ]:
from transformers import pipeline

def classify_document(text):
    CATEGORIES = ["Bank Statement", "Pay Slip", "Appraisal Report", "Lender Fees Worksheet", "Contract"]
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    result = classifier(text[:1000], CATEGORIES)  # Limit to first 1000 chars

    return result["labels"][0]

In [ ]:
def process_and_index_pdf(classified_docs):
    """Load a PDF and convert it to LlamaIndex Document format using PyMuPDF."""

    # Extract text from each page
    index_map = {}
    semantic_splitter = SemanticSplitterNodeParser(llm=llm, embed_model=embed_model, breakpoint_threshold_type="relative", breakpoint_threshold=0.1)

    for doc_id, data in classified_docs.items():
        doc_type = data["doc_type"]

        document = Document(text=data["text"], metadata={"doc_type": doc_type})
        if doc_type not in index_map:
            index_map[doc_type] = VectorStoreIndex.from_documents([document], embed_model=embed_model, node_parser=semantic_splitter)
        else:
            index_map[doc_type].insert(document)

        print(f"Indexed {doc_id} as {doc_type}.")
    print(index_map)

    return index_map

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle
from llama_index.core.response_synthesizers import CompactAndRefine
import re

def build_rag_pipeline(index_map, query):
    """Build a retrieval-augmented generation (RAG) pipeline."""

    CATEGORIES = ["Bank Statement", "Pay Slip", "Appraisal Report", "Lender Fees Worksheet", "Contract"]
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    result = classifier(text[:1000], CATEGORIES)  # Limit text to 1000 characters for efficiency
    doc_type = result["labels"][0]  # Return the highest confidence label

    if doc_type not in CATEGORIES:
        doc_type = "Unknown"
    if doc_type in index_map:
        print(f"Query classified as '{doc_type}', using its index.")
        selected_indices = [index_map[doc_type]]
    else:
        print("Query classification uncertain, using all available documents.")
        selected_indices = list(index_map.values())  # Use all indices if unknown

    # Step 3: Gather all nodes from selected indices
    all_nodes = []
    for index in selected_indices:
        all_nodes.extend(index.docstore.docs.values())

    if not all_nodes:
        return "No documents found to process query."

    num_nodes = len(all_nodes)
    safe_top_k = min(2, max(1, num_nodes))

    print(f"Using {num_nodes} total nodes from {len(selected_indices)} indices, with top_k={safe_top_k}")

    # Step 4: Create hybrid retriever
    vector_retrievers = [index.as_retriever(similarity_top_k=safe_top_k) for index in selected_indices]
    keyword_retriever = BM25Retriever.from_defaults(nodes=all_nodes, similarity_top_k=safe_top_k)

    class HybridRetriever(BaseRetriever):
        """Hybrid retriever combining vector and keyword search results."""
        def __init__(self, vector_retrievers, keyword_retriever, top_k=2):
            self.vector_retrievers = vector_retrievers
            self.keyword_retriever = keyword_retriever
            self.top_k = top_k
            super().__init__()

        def _retrieve(self, query_bundle, **kwargs):
            """Retrieve from all vector retrievers and keyword retriever."""
            vector_nodes = []
            for retriever in self.vector_retrievers:
                vector_nodes.extend(retriever.retrieve(query_bundle))

            keyword_nodes = self.keyword_retriever.retrieve(query_bundle)

            # Combine results, remove duplicates
            unique_nodes = {node.node_id: node for node in vector_nodes + keyword_nodes}
            sorted_nodes = sorted(unique_nodes.values(), key=lambda x: x.score if hasattr(x, 'score') else 0.0, reverse=True)

            return sorted_nodes[:self.top_k]

    hybrid_retriever = HybridRetriever(vector_retrievers, keyword_retriever, top_k=safe_top_k)

    # Step 5: Use query fusion and reranking
    fusion_retriever = QueryFusionRetriever(
        retrievers=[hybrid_retriever],
        llm=llm,
        similarity_top_k=2,
        num_queries=3,
        mode="reciprocal_rerank"
    )

    # Step 6: Apply reranking if more than one document exists
    reranker = None
    if num_nodes > 1:
        reranker = SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-6-v2", top_n=min(2, num_nodes))

    node_postprocessors = [reranker] if reranker else []

    # Step 7: Build query engine
    response_synthesizer = CompactAndRefine(llm=llm, verbose=True)

    query_engine = RetrieverQueryEngine.from_args(
        retriever=fusion_retriever,
        llm=llm,
        node_postprocessors=node_postprocessors,
        response_synthesizer=response_synthesizer
    )

    return query_engine


In [ ]:
from llama_index.llms.llama_cpp import LlamaCPP

# Initialize Mistral LLM
nest_asyncio.apply()
llm = LlamaCPP(
    model_path="/content/mistral.gguf",
    temperature=0.0,
    max_new_tokens=512,
    context_window=4096,
    model_kwargs={"n_gpu_layers": 1}
)
Settings.llm = llm

# Initialize embedding model
embed_model = HuggingFaceEmbedding(model_name="intfloat/e5-base-v2") # sentence-transformers/all-MiniLM-L6-v2 - old model
Settings.embed_model = embed_model

pdf_path = upload_pdf()
# Classify each document
classified_docs = {}
doc_texts = extract_text_from_pdf(pdf_path)
for doc_id, text in doc_texts.items():
    doc_type = classify_document(text)
    classified_docs[doc_id] = {"text": text, "doc_type": doc_type}
    print(f"{doc_id} classified as: {doc_type}")

index_map = process_and_index_pdf(classified_docs)

llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from /content/mistral.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.attention.hea

Please select a PDF file to upload:


Saving appraisal_report.pdf to appraisal_report (13).pdf
Saving payslip_sample_image.pdf to payslip_sample_image (13).pdf
Saving sample_bank_statement.pdf to sample_bank_statement (13).pdf
Saving sample_contract.pdf to sample_contract (13).pdf
Saving LenderFeesWorksheetNew.pdf to LenderFeesWorksheetNew (13).pdf
PDF saved to sample_docs/appraisal_report (13).pdf
PDF saved to sample_docs/payslip_sample_image (13).pdf
PDF saved to sample_docs/sample_bank_statement (13).pdf
PDF saved to sample_docs/sample_contract (13).pdf
PDF saved to sample_docs/LenderFeesWorksheetNew (13).pdf
PDF: sample_docs/appraisal_report (13).pdf
Number of pages: 10
Extracted 6470 words from the PDF.
PDF: sample_docs/payslip_sample_image (13).pdf
Number of pages: 1
Extracted 82 words from the PDF.
PDF: sample_docs/sample_bank_statement (13).pdf
Number of pages: 1
Extracted 287 words from the PDF.
PDF: sample_docs/sample_contract (13).pdf
Number of pages: 2
Extracted 315 words from the PDF.
PDF: sample_docs/LenderFe

Device set to use cpu


Doc-1 classified as: Appraisal Report


...

Doc-3 classified as: Contract


Device set to use cpu


Doc-4 classified as: Contract


Device set to use cpu


Doc-5 classified as: Lender Fees Worksheet
Indexed Doc-1 as Appraisal Report.
Indexed Doc-2 as Pay Slip.
Indexed Doc-3 as Contract.
Indexed Doc-4 as Contract.
Indexed Doc-5 as Lender Fees Worksheet.
{'Appraisal Report': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7ba2933e9d10>, 'Pay Slip': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7ba293314f90>, 'Contract': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7ba293204350>, 'Lender Fees Worksheet': <llama_index.core.indices.vector_store.base.VectorStoreIndex object at 0x7ba29827b0d0>}


In [ ]:
query = "What is the net pay on the payslip?"
rag_engine = build_rag_pipeline(index_map, query)
print(rag_engine)
response = rag_engine.query(query)
print('\nFinal Response:\n ---------------------- \n')
print(response)

Device set to use cpu
DEBUG:bm25s:Building index from IDs objects


Query classified as 'Lender Fees Worksheet', using its index.
Using 1 total nodes from 1 indices, with top_k=1


llama_perf_context_print:        load time =    3502.02 ms
llama_perf_context_print: prompt eval time =    3501.77 ms /    59 tokens (   59.35 ms per token,    16.85 tokens per second)
llama_perf_context_print:        eval time =   12177.52 ms /   107 runs   (  113.81 ms per token,     8.79 tokens per second)
llama_perf_context_print:       total time =   15746.04 ms /   166 tokens
Llama.generate: 1 prefix-match hit, remaining 1171 prompt tokens to eval
llama_perf_context_print:        load time =    3502.02 ms
llama_perf_context_print: prompt eval time =   87831.50 ms /  1171 tokens (   75.01 ms per token,    13.33 tokens per second)
llama_perf_context_print:        eval time =    1195.12 ms /     9 runs   (  132.79 ms per token,     7.53 tokens per second)
llama_perf_context_print:       total time =   89032.53 ms /  1180 tokens



Final Response:
 ---------------------- 

1,121.53

